# Domain Driver Data Mining Project

In [27]:
import numpy as np
import pandas as pd
import re
import os
import pickle
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MultiLabelBinarizer
from skmultilearn.problem_transform import LabelPowerset
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score, silhouette_score

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier

import warnings

from xgboost import XGBClassifier

warnings.filterwarnings("ignore")

In [2]:

train_preprocessed_file_path ="../dataset/train_data_preprocessed_1130pm_12_12.csv"
test_preprocessed_file_path ="../dataset/test_data_preprocessed_11pm_12_12.csv"
articles_testing_file_path = "../dataset/articles_testing.tsv"



In [3]:
df_preprocessed_data = pd.read_csv(train_preprocessed_file_path)
print(df_preprocessed_data.shape)
print(df_preprocessed_data.describe())
df_preprocessed_data.head()

(97846, 5)
          tag_count
count  97846.000000
mean      19.969033
std        0.507697
min        4.000000
25%       20.000000
50%       20.000000
75%       20.000000
max       23.000000


,content,tags,preprocessed_content,preprocessed_tags,tag_count
0,"Bước vào trận đấu, Barcelona nhanh chóng tràn ...","La Liga,Sevilla,Olimpic Lluis Companys,đè bẹp,...",trận đấu barcelona nhanh_chóng tràn tấn_công v...,"la_liga,sevilla,olimpic_lluis_companys,đè_bẹp,...",20
1,Willian đi vào lịch sử bóng đá xứ samba. Với 1...,"Estevao Willian,Neymar,giải VĐQG Brazil,Serie ...",willian đi lịch_sử bóng_đá xứ samba 1 bàn thắn...,"estevao_willian,neymar,giải_vđqg_brazil,serie_...",20
2,Giải vô địch ná cao su thế giới năm 2024 đã di...,"ná cao su,giải vô địch,giải Ba,Thượng Hải,đồng...",giải vô_địch ná cao_su thế_giới năm 2024 diễn ...,"ná_cao_su,giải_vô_địch,giải_ba,thượng_hải,đồng...",19
3,Mục tiêu lớn Phó Thủ tướng Chính phủ Lê Thành ...,"đăng cai,Đại hội Thể thao châu Á,ASIAD,thể dục...",mục_tiêu phó thủ_tướng chính_phủ lê_thành_long...,"đăng_cai,đại_hội_thể_thao_châu_á,asiad,thể_dục...",20
4,"Ngày 20-10, Giải bơi và lặn vô địch quốc gia n...","toàn đoàn,lặn,Vũ Đặng Nhật Nam,Nguyễn Lê Truyề...",20 10 giải bơi lặn vô_địch quốc_gia năm 2024 k...,"toàn_đoàn,lặn,vũ_đặng_nhật_nam,nguyễn_lê_truyề...",20


In [5]:
df_train = df_preprocessed_data[['preprocessed_content', 'preprocessed_tags']]
print(df_train.shape)
df_train.head()

(97846, 2)


,preprocessed_content,preprocessed_tags
0,trận đấu barcelona nhanh_chóng tràn tấn_công v...,"la_liga,sevilla,olimpic_lluis_companys,đè_bẹp,..."
1,willian đi lịch_sử bóng_đá xứ samba 1 bàn thắn...,"estevao_willian,neymar,giải_vđqg_brazil,serie_..."
2,giải vô_địch ná cao_su thế_giới năm 2024 diễn ...,"ná_cao_su,giải_vô_địch,giải_ba,thượng_hải,đồng..."
3,mục_tiêu phó thủ_tướng chính_phủ lê_thành_long...,"đăng_cai,đại_hội_thể_thao_châu_á,asiad,thể_dục..."
4,20 10 giải bơi lặn vô_địch quốc_gia năm 2024 k...,"toàn_đoàn,lặn,vũ_đặng_nhật_nam,nguyễn_lê_truyề..."


In [6]:
df_train['preprocessed_tags'] = df_train['preprocessed_tags'].str.split(',')

In [7]:
all_tags = [tag for tags in df_train['preprocessed_tags'] for tag in tags]
common_tags = [tag for tag, count in Counter(all_tags).most_common(6704)]

In [8]:
df_train['filtered_tags'] = df_train['preprocessed_tags'].apply(lambda tags: [tag for tag in tags if tag in common_tags])

In [9]:
df_train['filtered_tags'] = df_train['filtered_tags'].map(lambda x : ' '.join(x))

In [10]:
df_train.head()

,preprocessed_content,preprocessed_tags,filtered_tags
0,trận đấu barcelona nhanh_chóng tràn tấn_công v...,"[la_liga, sevilla, olimpic_lluis_companys, đè_...",la_liga lewandowski barcelona raphinha hansi_f...
1,willian đi lịch_sử bóng_đá xứ samba 1 bàn thắn...,"[estevao_willian, neymar, giải_vđqg_brazil, se...",neymar phá_kỷ_lục kiến_tạo gây_sốt tài_năng_trẻ
2,giải vô_địch ná cao_su thế_giới năm 2024 diễn ...,"[ná_cao_su, giải_vô_địch, giải_ba, thượng_hải,...",giải_vô_địch giải_ba thượng_hải đồng_đội vận_đ...
3,mục_tiêu phó thủ_tướng chính_phủ lê_thành_long...,"[đăng_cai, đại_hội_thể_thao_châu_á, asiad, thể...",đăng_cai thể_dục phấn_đấu chiến_lược_phát_triể...
4,20 10 giải bơi lặn vô_địch quốc_gia năm 2024 k...,"[toàn_đoàn, lặn, vũ_đặng_nhật_nam, nguyễn_lê_t...",toàn_đoàn đồng_nai huy_chương_bạc huy_chương_v...


In [11]:
tags = df_train['filtered_tags'].values
tags = [[l for l in clean.split()] for clean in tags]

In [12]:
# one hot encoding label
multilabel_binarizer = MultiLabelBinarizer(sparse_output=True)
multilabel_binarizer.fit(tags)
y = multilabel_binarizer.transform(tags)
y


<97846x6704 sparse matrix of type '<class 'numpy.int64'>'
	with 1034101 stored elements in Compressed Sparse Row format>

In [13]:
# show all of label unique
tags_list = multilabel_binarizer.classes_
tags_list

array(["'em", '24k', '2g', ..., 'ứng_trực', 'ứng_tuyển', 'ứng_viên'],
      dtype=object)

In [14]:
X = df_train['preprocessed_content']
X

0        trận đấu barcelona nhanh_chóng tràn tấn_công v...
1        willian đi lịch_sử bóng_đá xứ samba 1 bàn thắn...
2        giải vô_địch ná cao_su thế_giới năm 2024 diễn ...
3        mục_tiêu phó thủ_tướng chính_phủ lê_thành_long...
4        20 10 giải bơi lặn vô_địch quốc_gia năm 2024 k...
                               ...                        
97841    động_lực phát_triển bền_vững kinh_tế nhiệm_vụ ...
97842    số_liệu tổng_cục hải_quan 9 đầu năm 2024 xuất_...
97843    mong chiến_lược ứng_phó hiệu_quả thiên_tai kỳ ...
97844    nhà_chức_trách xác_định hoàng_văn_thảo hành_vi...
97845    6 đầu năm 2024 sở y_tế thành_phố đà_nẵng triển...
Name: preprocessed_content, Length: 97846, dtype: object

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=23)
type(X_train), type(X_test), type(y_train), type(y_test)

(pandas.core.series.Series,
 pandas.core.series.Series,
 scipy.sparse._csr.csr_matrix,
 scipy.sparse._csr.csr_matrix)

In [16]:
X_train.shape,y_train.shape

((88061,), (88061, 6704))

# Sử dụng mô hình học máy

## Sử dụng SGDC

In [18]:
# pipeline
clf_sgdc = Pipeline([("vectorizer",
                 TfidfVectorizer(max_features = 100000)),
                ("classifier", OneVsRestClassifier(SGDClassifier()))])
clf_sgdc.fit(X_train, y_train)
y_pred_sgdc = clf_sgdc.predict(X_test)


Accuracy : 0.002350536535513541
Macro f1 sample : 0.1706208501822783
Macro f1 score : 0.08448656141383039
Micro f1 score : 0.21409992921893523
Hamming loss : 0.0014048614422494125


In [30]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_sgdc))
print("F1 sample:",metrics.f1_score(y_test, y_pred_sgdc, average = 'samples'))
print("Macro f1:",metrics.f1_score(y_test, y_pred_sgdc, average = 'macro'))
print("Micro f1:",metrics.f1_score(y_test, y_pred_sgdc, average = 'micro'))
print("Hamming loss:",metrics.hamming_loss(y_test,y_pred_sgdc))

Accuracy: 0.002350536535513541
F1 sample: 0.1706208501822783
Macro f1: 0.08448656141383039
Micro f1: 0.21409992921893523
Hamming loss: 0.0014048614422494125


In [20]:
def predict_and_save_tags(model, input_csv, output_tsv):

    df_test = pd.read_csv(input_csv)
    df_result = pd.read_csv(articles_testing_file_path, sep='\t')

    X_test = df_test['preprocessed_content']
    y_pred_test = model.predict(X_test)
    predicted_tags = multilabel_binarizer.inverse_transform(y_pred_test)
    df_result['tags'] = [
        ",".join(tag.replace("_", " ") for tag in tags)
        for tags in predicted_tags
    ]

    # Save the DataFrame to a TSV file
    df_result.to_csv(output_tsv, sep='\t', index=False)



In [21]:
predict_and_save_tags(clf_sgdc, test_preprocessed_file_path, 'clf_sgdc_result.tsv')

In [22]:

model_result_path = "model_result"
file_name = "clf_sgdc.pkl"
file_path = os.path.join(model_result_path, file_name)
os.makedirs(model_result_path, exist_ok=True)
with open(file_path, "wb") as file:
    pickle.dump(clf_sgdc, file)
print(f"Mô hình đã được lưu tại: {file_path}")

Mô hình đã được lưu tại: model_result\clf_sgdc.pkl


In [23]:
clf_lr = Pipeline([("vectorizer", TfidfVectorizer(max_features = 100000)), ("classifier", OneVsRestClassifier(LogisticRegression(), n_jobs = 4))])
clf_lr.fit(X_train, y_train)
y_pred_lr = clf_lr.predict(X_test)



Accuracy : 0.002452733776188043
Macro f1 sample : 0.26700788656480173
Macro f1 score : 0.1910027212507754
Micro f1 score : 0.3340496730420901
Hamming loss : 0.0013227103488730864


In [31]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_lr))
print("f1 sample:",metrics.f1_score(y_test, y_pred_lr, average = 'samples'))
print("Macro f1:",metrics.f1_score(y_test, y_pred_lr, average = 'macro'))
print("Micro f1:",metrics.f1_score(y_test, y_pred_lr, average = 'micro'))
print("Hamming loss:",metrics.hamming_loss(y_test,y_pred_lr))

Accuracy: 0.002452733776188043
f1 sample: 0.26700788656480173
Macro f1: 0.1910027212507754
Micro f1: 0.3340496730420901
Hamming loss: 0.0013227103488730864


In [88]:
def predict_and_save_tags(model, input_csv, output_tsv):

    df_test = pd.read_csv(input_csv)
    df_result = pd.read_csv(articles_testing_file_path, sep='\t')

    X_test = df_test['preprocessed_content']
    y_pred_test = model.predict(X_test)
    predicted_tags = multilabel_binarizer.inverse_transform(y_pred_test)
    df_result['tags'] = [
        ",".join(tag.replace("_", " ") for tag in tags)
        for tags in predicted_tags
    ]

    # Save the DataFrame to a TSV file
    df_result.to_csv(output_tsv, sep='\t', index=False)



In [24]:
predict_and_save_tags(clf_lr, test_preprocessed_file_path, 'clf_lr_result.tsv')

In [25]:

model_result_path = "model_result"
file_name = "clf_lr.pkl"
file_path = os.path.join(model_result_path, file_name)
os.makedirs(model_result_path, exist_ok=True)
with open(file_path, "wb") as file:
    pickle.dump(clf_lr, file)
print(f"Mô hình đã được lưu tại: {file_path}")

Mô hình đã được lưu tại: model_result\clf_lr.pkl
